<a href="https://colab.research.google.com/github/mtzig/badwriting_OCR/blob/main/iam_baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
iam_path = '../drive/MyDrive/semester2/cs766/iam_dataset'

In [12]:
# %cd drive/MyDrive/semester2/cs766/iam_dataset

In [13]:
# %mkdir sentences
# !tar -xf sentences.tar -C sentences

In [14]:
# %mkdir lines
# !tar -xf lines.tar -C lines

In [15]:
# %mkdir ascii
# !tar -xf ascii.tar -C ascii

In [16]:
!git clone https://github.com/mtzig/badwriting_OCR.git
%cd /content/badwriting_OCR/

Cloning into 'badwriting_OCR'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 291 (delta 32), reused 91 (delta 28), pack-reused 192
Receiving objects: 100% (291/291), 8.21 MiB | 16.70 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (156/156), done.
/content/badwriting_OCR


In [17]:
!pip install -q transformers
!pip install -q datasets jiwer

In [18]:
from utils import model_utils
from transformers import AdamW
from tqdm.notebook import tqdm
import torch
from PIL import Image

/content/badwriting_OCR/utils/model_utils.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a t

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Code to generate csv file with iam file names and paths

In [20]:
# make a csv file with the file names and text starting from directory sentences
# Filename                       Text
# a01/a01-000u/a01-000u-00.png   A MOVE to stop...

# import csv

# data = [['file_name', 'text']]

# with open(iam_path + '/ascii/lines.txt') as file:
#   for line in file:
#     if line[0] != '#': # ignore instructions
#       file_name = line.split()[0]

#       dash1 = file_name.find('-')
#       dash2 = file_name.find('-', dash1+1)

#       file_name = file_name[:dash1]+'/'+file_name[:dash2]+'/'+file_name+'.png'

#       text = line.split()[-1]
#       text = ' '.join(text.split('|'))
#       data.append([file_name, text])

# csv_file = iam_path + '/iam_data.csv'

# with open(csv_file, mode='w', newline='') as file:
#   writer = csv.writer(file)
#   writer.writerows(data)

In [21]:
train_dataloader,eval_dataloader = model_utils.get_dataloaders(dataset_type='iam', root=iam_path)
model = model_utils.getModel(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import TrOCRProcessor
import pandas as pd

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")

def compute_outputs(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)


    return pred_str, label_str

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [ ]:
base_preds, base_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    base_preds.extend(p)
    base_labels.extend(l)

base_df = pd.DataFrame(data={'preds':base_preds, 'lables':base_labels})
base_df.to_csv('base.csv', index=False)




  0%|          | 0/668 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [ ]:
full_train_cer = []

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = model_utils.compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer

   full_train_cer.append(cer)

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

In [ ]:
full_preds, full_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    full_preds.extend(p)
    full_labels.extend(l)

full_df = pd.DataFrame(data={'preds':full_preds, 'lables':full_labels})
full_df.to_csv('full.csv', index=False)

In [ ]:
model = model_utils.getModel(device)
# for pn, p in model.named_parameters():
#   if pn != 'decoder.output_projection.weight':
#     p.requires_grad = False

for pn, p in model.named_parameters():
  if pn.startswith('encoder'):
    p.requires_grad = False

In [ ]:
freeze_train_cer = []

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):  # loop over the dataset multiple times
   # train
   model.train()
   train_loss = 0.0
   for batch in tqdm(train_dataloader):
      # get the inputs
      for k,v in batch.items():
        batch[k] = v.to(device)

      # forward + backward + optimize
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss.item()

   print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

   # evaluate
   model.eval()
   valid_cer = 0.0
   with torch.no_grad():
     for batch in tqdm(eval_dataloader):
       # run batch generation
       outputs = model.generate(batch["pixel_values"].to(device))
       # compute metrics
       cer = model_utils.compute_cer(pred_ids=outputs, label_ids=batch["labels"])
       valid_cer += cer

   freeze_train_cer.append(cer)

   print("Validation CER:", valid_cer / len(eval_dataloader))

model.save_pretrained(".")

In [ ]:
freeze_preds, freeze_labels = [], []
with torch.no_grad():
  for batch in tqdm(eval_dataloader):
    # run batch generation
    outputs = model.generate(batch["pixel_values"].to(device))
    p, l = compute_outputs(outputs, batch["labels"].to(device))
    freeze_preds.extend(p)
    freeze_labels.extend(l)

freeze_df = pd.DataFrame(data={'preds':freeze_preds, 'lables':freeze_labels})
freeze_df.to_csv('freeze.csv', index=False)

In [ ]:
cer_df = pd.DataFrame(data={'full':full_train_cer, 'freeze':freeze_train_cer})
cer_df.to_csv('cer.csv', index=False)